<a href="https://colab.research.google.com/github/616055/pdp_assignment_3_616055/blob/master/PDP_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup environment for Spark**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

**Load data and map the columns with Spark**

In [12]:
# import packages
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

# start the session
findspark.init()
spark = SparkSession.builder \
            .master("local") \
            .appName("Titanic") \
            .config("spark.executor.memory", "1gb") \
            .getOrCreate()
sc = spark.sparkContext

# load the titanic text file
rdd = sc.textFile("titanic.csv")
# get the header line (first line)
header = rdd.first()
# get all rows which are not the header
rdd = rdd.filter(lambda line: line != header)
# split the values by a comma
rdd = rdd.map(lambda line: line.split(","))
# define the headers
df = rdd.map(lambda line: Row(
    survived=line[0],
    pclass=line[1],
    name=line[2],
    sex=line[3],
    age=line[4],
    siblings_spoused_aboard=line[5],
    parents_children_aboard=line[6],
    fare=line[7]
)).toDF()
# convert age and fare values to integers
df = df.withColumn("age", df["age"].cast(IntegerType()))
df = df.withColumn("fare", df["fare"].cast(FloatType()))

**Assignment 3a**
*Calculate the conditional probability that a person survives given their sex and passenger-class*

In [13]:
def calculateProbability(df, sex, pclass):
  # where conditions for sex and passenger class
  filtered = df.rdd.filter(lambda line: line['sex'] == sex and line['pclass'] == pclass)

  # get the amount of survived and not survived persons
  survived = filtered.filter(lambda line: line['survived'] == '1').collect()
  not_survived = filtered.filter(lambda line: line['survived'] == '0').collect()

  # convert it back to a dataframe
  survived_df = spark.createDataFrame(survived)
  not_survived_df = spark.createDataFrame(not_survived)

  # get the amount of persons for survived and the total persons
  survived_persons = survived_df.count()
  total_persons = survived_persons + not_survived_df.count()

  return survived_persons, total_persons

In [14]:
# P( S = true | G = female, C = 1 )
df1 = calculateProbability(df, 'female', '1')
percentage1 = df1[0] / df1[1] * 100
print("Conditional probability of P( S = true | G = female, C = 1 ) = " + str(df1[0]) + " / " + str(df1[1]) + " = " + str(percentage1) + "%")

# P( S = true | G = female, C = 2 )
df2 = calculateProbability(df, 'female', '2')
percentage2 = df2[0] / df2[1] * 100
print("Conditional probability of P( S = true | G = female, C = 2 ) = " + str(df2[0]) + " / " + str(df2[1]) + " = " + str(percentage2) + "%")

# P( S = true | G = female, C = 2 )
df3 = calculateProbability(df, 'female', '3')
percentage3 = df3[0] / df3[1] * 100
print("Conditional probability of P( S = true | G = female, C = 3 ) = " + str(df3[0]) + " / " + str(df3[1]) + " = " + str(percentage3) + "%")

# P( S = true | G = male, C = 1 )
df4 = calculateProbability(df, 'male', '1')
percentage4 = df4[0] / df4[1] * 100
print("Conditional probability of P( S = true | G = male, C = 1 ) = " + str(df4[0]) + " / " + str(df4[1]) + " = " + str(percentage4) + "%")

# P( S = true | G = male, C = 2 )
df5 = calculateProbability(df, 'male', '2')
percentage5 = df5[0] / df5[1] * 100
print("Conditional probability of P( S = true | G = male, C = 2 ) = " + str(df5[0]) + " / " + str(df5[1]) + " = " + str(percentage5) + "%")

# P( S = true | G = male, C = 3 )
df6 = calculateProbability(df, 'male', '3')
percentage6 = df6[0] / df6[1] * 100
print("Conditional probability of P( S = true | G = male, C = 3 ) = " + str(df6[0]) + " / " + str(df6[1]) + " = " + str(percentage6) + "%")

Conditional probability of P( S = true | G = female, C = 1 ) = 91 / 94 = 96.80851063829788%
Conditional probability of P( S = true | G = female, C = 2 ) = 70 / 76 = 92.10526315789474%
Conditional probability of P( S = true | G = female, C = 3 ) = 72 / 144 = 50.0%
Conditional probability of P( S = true | G = male, C = 1 ) = 45 / 122 = 36.885245901639344%
Conditional probability of P( S = true | G = male, C = 2 ) = 17 / 108 = 15.74074074074074%
Conditional probability of P( S = true | G = male, C = 3 ) = 47 / 343 = 13.702623906705538%


Conditional probability of P( S = true | G = female, C = 1 ) = 91 / 94 = 96.8%<br/>
Conditional probability of P( S = true | G = female, C = 2 ) = 70 / 76 = 92.1%<br/>
Conditional probability of P( S = true | G = female, C = 3 ) = 72 / 144 = 50.0%<br/>
Conditional probability of P( S = true | G = male, C = 1 ) = 45 / 122 = 36.9%<br/>
Conditional probability of P( S = true | G = male, C = 2 ) = 17 / 108 = 15.7%<br/>
Conditional probability of P( S = true | G = male, C = 3 ) = 47 / 343 = 13.7%

**Assignment 3b**
*What is the probability that a child who is in third class and is 10 years old or younger survives?*

In [15]:
# where conditions for age and passenger class
filtered = df.rdd.filter(lambda line: line['age'] <= 10 and line['pclass'] == '3')

# get the amount of survived and not survived persons
survived = filtered.filter(lambda line: line['survived'] == '1').collect()
not_survived = filtered.filter(lambda line: line['survived'] == '0').collect()

# convert it back to a dataframe
survived_df = spark.createDataFrame(survived)
not_survived_df = spark.createDataFrame(not_survived)

# get the amount of persons for survived and the total persons
survived_persons = survived_df.count()
not_survived_persons = not_survived_df.count()
total_persons = survived_persons + not_survived_df.count()

percentage = survived_persons / total_persons * 100

In [ ]:
# P( S = true | A ≤ 10, C = 3 )
print("P( S = true | A ≤ 10, C = 3 ) = " + str(survived_persons) + " / " + str(total_persons) + " = " + str(percentage) + "%")
print("Beta(⍺=" + str(survived_persons) + ", β=" + str(not_survived_persons) + ")")

P( S = true | A ≤ 10, C = 3 ) = 22 / 53 = 41.5%<br/>
Beta(⍺=22, β=31)

**Assignment 3c**
*How much did people pay to be on the ship?*

In [ ]:
def calculateExpectation(df, pclass):
  # where condition for passenger class
  filtered = df.rdd.filter(lambda line: line['pclass'] == pclass)
  # convert it back to a dataframe
  filtered_df = spark.createDataFrame(filtered)
  # calculate the average of the paid fare
  expectation = filtered_df.groupBy('pclass').avg().collect()

  return expectation

In [ ]:
# E[ X | C = 1 ]
expectation1 = calculateExpectation(df, '1')
print("E[ X | C = 1 ] = " + str(expectation1[0][2]) + " pounds")

# E[ X | C = 2 ]
expectation2 = calculateExpectation(df, '2')
print("E[ X | C = 2 ] = " + str(expectation2[0][2]) + " pounds")

# E[ X | C = 3 ]
expectation3 = calculateExpectation(df, '3')
print("E[ X | C = 3 ] = " + str(expectation3[0][2]) + " pounds")

E[ X | C = 1 ] = 84.15 pounds<br/>
E[ X | C = 2 ] = 20.66 pounds<br/>
E[ X | C = 3 ] = 13.71 pounds

**End session**

In [ ]:
spark.stop()